In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import pandas as pd
from tqdm import tqdm
from datetime import datetime

In [2]:
driver = webdriver.Chrome()

In [3]:
urls_city = {"Centro": "https://www.vivareal.com.br/aluguel/sp/campinas/bairros/centro/apartamento_residencial/",
             "Cambui": "https://www.vivareal.com.br/aluguel/sp/campinas/bairros/cambui/apartamento_residencial/",
             "Barao Geraldo": "https://www.vivareal.com.br/aluguel/sp/campinas/bairros/barao-geraldo/apartamento_residencial/",
             "Vila Insdutrial": "https://www.vivareal.com.br/aluguel/sp/campinas/bairros/vila-industrial/apartamento_residencial/",
             "Jardim Itatiaia": "https://www.vivareal.com.br/aluguel/sp/campinas/bairros/jardim-itatiaia/apartamento_residencial/",
             "Botafogo": "https://www.vivareal.com.br/aluguel/sp/campinas/bairros/botafogo/apartamento_residencial/",
             "Taquaral": "https://www.vivareal.com.br/aluguel/sp/campinas/bairros/taquaral/apartamento_residencial/",
             "Vila Itapura": "https://www.vivareal.com.br/aluguel/sp/campinas/bairros/vila-itapura/apartamento_residencial/",
             "Bosque": "https://www.vivareal.com.br/aluguel/sp/campinas/bairros/bosque/apartamento_residencial/", 
             "Jardim Guanabara": "https://www.vivareal.com.br/aluguel/sp/campinas/bairros/jardim-guanabara/apartamento_residencial/",
             "Alpha Ville Campinas": "https://www.vivareal.com.br/aluguel/sp/campinas/bairros/loteamento-alphaville-campinas/apartamento_residencial/",
             "Parque das Universidades": "https://www.vivareal.com.br/aluguel/sp/campinas/bairros/parque-das-universidades/apartamento_residencial/",
             "Bonfim": "https://www.vivareal.com.br/aluguel/sp/campinas/bairros/bonfim/apartamento_residencial/"}

In [4]:
urls_dict = {"Campinas": "https://www.vivareal.com.br/aluguel/sp/campinas/apartamento_residencial/"}

errors = []
results = []

for bairro, url in urls_dict.items():
    current_url = url
    driver.get(url)
    sleep(2)
    actions = ActionChains(driver)

    try:
        driver.find_element_by_class_name("cookie-notifier__cta").click()
    except:
            print("No cookies!")
    
    for i in tqdm(range(5), desc=bairro):
        
        
        sleep(5)
        main_div = driver.find_element_by_class_name("results-main__panel")
        properties = main_div.find_elements_by_class_name("js-property-card")
        paginator = driver.find_element_by_class_name("js-results-pagination")
        next_page = paginator.find_element_by_xpath("//a[@title='Próxima página']") #/html/body/main/div[2]/div[1]/section/div[2]/div[2]/div/ul/li[9]/a
        
        for i,apartment in enumerate(properties):
            url = apartment.find_element_by_class_name("js-card-title").get_attribute("href")
            apto_id = url.split("id-")[-1][:-1]
            header = apartment.find_element_by_class_name("property-card__title").text
            address = apartment.find_element_by_class_name("property-card__address").text
            area = apartment.find_element_by_class_name("js-property-card-detail-area").text
            rooms = apartment.find_element_by_class_name("js-property-detail-rooms").text
            bathrooms = apartment.find_element_by_class_name("js-property-detail-bathroom").text
            garages = apartment.find_element_by_class_name("js-property-detail-garages").text
            try:
                amenities = apartment.find_element_by_class_name("property-card__amenities").text
            except:
                amenities = None
                errors.append(url)
            price = apartment.find_element_by_class_name("js-property-card-prices").text
            try:
                condo = apartment.find_element_by_class_name("js-condo-price").text
            except:
                condo = None
                errors.append(url)
            crawler = bairro
            crawled_at = datetime.now().strftime("%Y-%m-%d %H:%M")
            results.append({"id": apto_id,
                            "url": url,
                            "header": header,
                            "address": address,
                            "area": area,
                            "rooms": rooms,
                            "bathrooms": bathrooms,
                            "garages": garages,
                            "amenities": amenities,
                            "price": price,
                            "condo": condo,
                            "crawler": crawler,
                            "crawled_at": crawled_at})
        next_page.click()
pd.DataFrame(results).to_csv("full_results.csv", index=False)
driver.close()

Campinas: 100%|██████████| 5/5 [01:08<00:00, 13.66s/it]


In [5]:
df = pd.DataFrame(results)

In [6]:
df.shape

(180, 13)

In [7]:
df.head()

,id,url,header,address,area,rooms,bathrooms,garages,amenities,price,condo,crawler,crawled_at
0,2558558970,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com Quarto para Aluguel, 49m²","Rua Itu, 86 - Centro, Campinas - SP",49,1 Quarto,2 Banheiros,1 Vaga,Portão eletrônico,R$ 950 /mês,R$ 583,Campinas,2022-05-11 22:33
1,2541694419,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 3 Quartos para Aluguel, 76m²","Avenida João Scarparo Netto, 240 - Loteamento ...",76,3 Quartos,2 Banheiros,2 Vagas,Piscina\nChurrasqueira\nElevador\nCondomínio f...,R$ 490.000 /mês,R$ 472,Campinas,2022-05-11 22:33
2,2559659498,https://www.vivareal.com.br/imovel/apartamento...,Apartamento com 2 Quartos para Venda/Aluguel 75m²,"Rua Hermantino Coelho, 161 - Mansões Santo Ant...",75,2 Quartos,2 Banheiros,1 Vaga,Varanda,R$ 2.600 /mês\nR$ 480.000,R$ 600,Campinas,2022-05-11 22:33
3,2561701454,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 3 Quartos para Aluguel, 103m²","Avenida Wellman Galvão de França Rangel, 798 -...",103,3 Quartos,3 Banheiros,2 Vagas,Piscina\nChurrasqueira\nElevador\nAcademia\nPl...,R$ 3.500 /mês,R$ 1.000,Campinas,2022-05-11 22:33
4,2561568233,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 2 Quartos para Aluguel, 72m²","Rua Serra dos Itatins - Jardim Proença, Campin...",72,2 Quartos,1 Banheiro,1 Vaga,Cozinha\nElevador\nInterfone\nLavanderia,R$ 750 /mês,R$ 650,Campinas,2022-05-11 22:33


In [8]:
df.describe()

,id,url,header,address,area,rooms,bathrooms,garages,amenities,price,condo,crawler,crawled_at
count,180,180,180,180,180,180,180,180,157,180,169,180,180
unique,177,177,120,141,77,4,5,5,124,104,126,1,2
top,2558023464,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 2 Quartos para Aluguel, 70m²","Cambuí, Campinas - SP",75,2 Quartos,2 Banheiros,1 Vaga,Piscina\nChurrasqueira\nElevador\nCondomínio f...,R$ 2.000 /mês,R$ 600,Campinas,2022-05-11 22:33
freq,2,2,6,8,12,74,77,113,8,8,6,180,146
